In [9]:

import torch
from torch import nn
from d2l import torch as d2l
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset


from torch.utils.data import DataLoader, Dataset
from inferno.io.transform.base import Transform, Compose
from inferno.io.transform.generic import Normalize, AsTorchBatch
from inferno.io.transform.image import RandomCrop, RandomRotate, RandomFlip
from libtiff import TIFF
import os
import torch
import numpy as np


试打开tiff


第一种读取TIff的方法


In [2]:
from PIL import Image
data = Image.open("data/Sample2/samples.tif")

print(data.format, data.size, data.mode)
print(32*"-")
print(data.info)


第二种方式打开tiff


In [11]:
tif = TIFF.open('data/Sample2/samples.tif', mode='r')
tifLst = list(tif.iter_images()) 
tifArr = np.zeros((len(tifLst), tifLst[0].shape[0], tifLst[0].shape[1]))


第三种打开tiff的方法


In [6]:
import cv2

img = cv2.imread('data/Sample2/samples.tif')



In [8]:
img1 = torch.from_numpy(img)


In [12]:
## Read the data

class MyDataSet(Dataset):
    # Parameters and their form vary according to program needs
    def __init__(self, pathLst, transform):
        dataPath, labelPath = pathLst
        self.tifStreamData, self.tifStreamLabel = [], []
        dataFiles, labelFiles = os.listdir(dataPath), os.listdir(labelPath)
        # sorted by name order, such as LR_20.tif
        dataFiles.sort(key=lambda x: int(x[3:-4]))
        for dataFile in dataFiles:
            dataFileName = os.path.join(dataPath, dataFile)
            self.tifStreamData.append(tiff2Stack(dataFileName, transform))

        labelFiles.sort(key=lambda x: int(x[3:-4]))
        for labelFile in labelFiles:
            labelFileName = os.path.join(labelPath, labelFile)
            self.tifStreamLabel.append(tiff2Stack(labelFileName, transform))
        assert len(self.tifStreamData) == len(
            self.tifStreamLabel)    # check length

    def __len__(self):
        return len(self.tifStreamData)

    def __getitem__(self, idx):
        data, label = self.tifStreamData, self.tifStreamLabel
        return data[idx], label[idx]


def tiff2Stack(fileName, transform=None):  # read tif, data transform, output tensor
    tif = TIFF.open(fileName, mode='r')
    tifLst = list(tif.iter_images())  # (51,101,101)
    tifArr = np.zeros((len(tifLst), tifLst[0].shape[0], tifLst[0].shape[1]))

    for i, img in enumerate(list(tif.iter_images())):
        # avoid that "can't convert np.ndarray of type numpy.uint16."
        tifArr[i, :, :] = img/1.0
    if transform:
        tifArr = transform(tifArr)
    return tifArr


In [13]:
data_read = tiff2Stack("data/Sample2/samples.tif")